# PyTorch: Optimization & Training

https://github.com/pytorch/examples/tree/master/mnist

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Preprocess

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

# Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization

- Model과 Optimization 설정

In [6]:
model = Net().to(device)

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [9]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [13]:
model.train() # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [14]:
data, target = next(iter(train_loader))

In [15]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [16]:
data, target = data.to(device), target.to(device)

In [17]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [18]:
optimizer.zero_grad()

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [19]:
output = model(data)

- Model에서 예측한 결과를 Loss Function에 넣음
    - 여기 예제에서는 Negative Log-Likelihood Loss 라는 Loss Function을 사용

In [20]:
loss = F.nll_loss(output, target)

- Back Propagation을 통해 Gradients를 계산

In [21]:
loss.backward()

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [22]:
optimizer.step()

# Start Training

위의 최적화 과정을 반복하여 학습 시작

In [23]:
epochs = 1
log_interval = 100

In [27]:
for epoch in range(1, epochs+1):
    #train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} {:,.0f}%]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    print('Training end...')

Train Epoch: 1 [0/60000 0%]	Loss: 0.350869
Train Epoch: 1 [6400/60000 11%]	Loss: 0.430692
Train Epoch: 1 [12800/60000 21%]	Loss: 0.293032
Train Epoch: 1 [19200/60000 32%]	Loss: 0.303908
Train Epoch: 1 [25600/60000 43%]	Loss: 0.271252
Train Epoch: 1 [32000/60000 53%]	Loss: 0.221334
Train Epoch: 1 [38400/60000 64%]	Loss: 0.406929
Train Epoch: 1 [44800/60000 75%]	Loss: 0.160506
Train Epoch: 1 [51200/60000 85%]	Loss: 0.291229
Train Epoch: 1 [57600/60000 96%]	Loss: 0.137454
Training end...


# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [28]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [30]:
test_loss = 0
correct = 0

with torch.no_grad():
    data,target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()
    

In [31]:
pred

tensor([[1],
        [9],
        [8],
        [6],
        [4],
        [0],
        [9],
        [6],
        [7],
        [3],
        [2],
        [2],
        [5],
        [5],
        [8],
        [5],
        [1],
        [5],
        [8],
        [7],
        [1],
        [8],
        [7],
        [5],
        [4],
        [7],
        [3],
        [5],
        [1],
        [8],
        [1],
        [3],
        [9],
        [7],
        [1],
        [8],
        [1],
        [7],
        [8],
        [0],
        [6],
        [4],
        [2],
        [4],
        [2],
        [1],
        [2],
        [9],
        [2],
        [6],
        [2],
        [5],
        [8],
        [7],
        [0],
        [3],
        [8],
        [1],
        [3],
        [1],
        [3],
        [7],
        [1],
        [1]])

In [32]:
test_loss

8.505084991455078

In [33]:
output

tensor([[-1.0254e+01, -9.0420e-03, -6.5606e+00, -7.5774e+00, -9.3938e+00,
         -7.2770e+00, -6.1380e+00, -7.5841e+00, -5.6661e+00, -8.9072e+00],
        [-8.5098e+00, -1.1728e+01, -7.2284e+00, -8.3469e+00, -2.4412e+00,
         -9.1402e+00, -8.8956e+00, -3.5018e+00, -7.2390e+00, -1.2708e-01],
        [-1.0711e+01, -1.0626e+01, -8.9055e+00, -7.2756e+00, -8.2785e+00,
         -4.7414e+00, -8.3619e+00, -1.0530e+01, -1.1049e-02, -7.0436e+00],
        [-8.6246e+00, -1.6167e+01, -5.8755e+00, -1.4810e+01, -7.6304e+00,
         -7.4065e+00, -4.3654e-03, -1.6412e+01, -8.3640e+00, -1.0071e+01],
        [-1.1966e+01, -1.9061e+01, -1.0615e+01, -1.4001e+01, -9.9633e-04,
         -1.0583e+01, -7.5012e+00, -1.3297e+01, -8.7832e+00, -8.3718e+00],
        [-2.5558e-02, -1.3615e+01, -4.8333e+00, -7.6530e+00, -1.0996e+01,
         -5.2406e+00, -5.0184e+00, -8.9264e+00, -5.5138e+00, -7.2574e+00],
        [-5.6799e+00, -1.1548e+01, -8.2430e+00, -1.0253e+01, -1.8347e+00,
         -6.4526e+00, -9.1644e+0

In [34]:
correct

63

In [35]:
target

tensor([1, 9, 8, 6, 4, 0, 9, 6, 7, 3, 2, 2, 5, 5, 8, 5, 1, 5, 8, 7, 1, 8, 7, 5,
        4, 7, 3, 5, 1, 8, 1, 3, 9, 7, 1, 8, 1, 7, 8, 0, 6, 4, 2, 4, 2, 1, 2, 9,
        2, 6, 2, 5, 8, 7, 0, 3, 8, 1, 5, 1, 3, 7, 1, 1])

In [39]:
pred.eq(target.view_as(pred)).sum().item()/64

0.984375

In [40]:
test_loss

8.505084991455078

In [41]:
correct

63

In [42]:
test_loss /= len(test_loader.dataset)

In [43]:
test_loss

0.0008505084991455078

In [44]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))


Test set: Average Loss: 0.2251, Accuracy: 9335/10000 (93%)

